In [17]:
# DON'T CHANGE THE IMPORTS ORDER
import glob
import xarray as xr
import geopandas as gpd

import panel as pn
pn.extension('plotly')

import hvplot.pandas  # noqa
import hvplot.xarray
from utils import utils 

import echopype as ep
import plotly.graph_objs as go

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [18]:
# DON'T DELETE THIS PART
from bokeh.settings import settings
settings.resources = 'cdn'
settings.resources = 'inline'

### Utils

A couple of functions created:

- **combine_data**

This function takes in all the \*.nc files and combines them. The problem with `ep.combine_data` function is that it doesn't catch the errors when given inappropriate data formats. That is especially evident if there are files like `Summer2017-D20170625-T190753_Sv.nc` (**that end with _Sv**) and `Summer2017-D20170625-T190753.nc`. 

For my purpose, I've created a separate function that ignores files with no latitude/longitude and combines all of the rest. For now, there are no explicit tests created, it's just a working prototype for the current data in hand. I'll be using lat/lon to create a GeoPandas & further create visualizations for maps and frequencies. 

- **get_geo & compute_mvbs**

The `get_geo` function creates a DataFrame that holds lat/lon as used in <a href="https://github.com/OSOceanAcoustics/echopype-examples/blob/main/notebooks/echopype_tour.ipynb">echopype_tour</a>. The other takes advantage of the `ep.calibrate.compute_Sv` functionality but created as a function to overcome redundancy. 

- **show_freq**

This function shows the maps depending on the selected frequencies. 

- **get_map**

This function creates a map that holds all of the traces using Plotly. Ideally, this function will have the capabilities to select using both box-select and lasso-select. For now, the function simply shows the map interactively. 

In [3]:
def get_map(df):
    """
    Parameters
    ----------
    df :
        return:

    Returns
    -------
    
    """
    lat, lon = geo_df['latitude'].median(), geo_df['longitude'].median()
    fig = go.Figure(go.Scattermapbox(
                    fill='toself',
                    lon=geo_df['longitude'],
                    lat=geo_df['latitude'])
                   )

    fig.update_layout(
                    mapbox={
                        'style': 'stamen-terrain',
                        'zoom': 7,
                        'center': {'lon': lon, 'lat': lat}
                    },
                    showlegend=False
    )

    fig['layout'].update(height=450, width=700)
    return fig.to_dict(), fig


### Get data

In [4]:
# get the converted .nc files and combine according to the
# presence of latitude/longitude. Otherwise, ignore the files

files = utils.combine_data()
files

Skipping file data\Summer2017-D20170625-T190753_Sv.nc since it does not have latitude/longitude


EchoData: standardized raw data from data\Summer2017-D20170728-T181619.nc
  > top: (Top-level) contains metadata about the SONAR-netCDF4 file format.
  > environment: (Environment) contains information relevant to acoustic propagation through water.
  > platform: (Platform) contains information about the platform on which the sonar is installed.
  > nmea: (Platform/NMEA) contains information specific to the NMEA protocol.
  > provenance: (Provenance) contains metadata about how the SONAR-netCDF4 version of the data were obtained.
  > sonar: (Sonar) contains specific metadata for the sonar system.
  > beam: (Beam) contains backscatter data and other beam or channel-specific data.
  > vendor: (Vendor specific) contains vendor-specific information about the sonar and the data.

In [5]:
# create a separate file to hold all the computed 
# frequencies for non-cluttered access

files_mvbs = utils.compute_mvbs(files)
files_mvbs

<xarray.Dataset>
Dimensions:    (ping_time: 391, frequency: 3, range: 150)
Coordinates:
  * ping_time  (ping_time) datetime64[ns] 2017-07-28T18:16:00 ... 2017-07-28T...
  * frequency  (frequency) float64 1.8e+04 3.8e+04 1.2e+05
  * range      (range) float64 0.0 5.0 10.0 15.0 ... 730.0 735.0 740.0 745.0
Data variables:
    Sv         (frequency, ping_time, range) float64 1.503 -73.88 ... -51.14
Attributes:
    binning_mode:          physical units
    range_meter_interval:  5m
    ping_time_interval:    20s

In [6]:
# create geopandas geometry points from lat/lon 

geo_df = utils.get_geo(files)
geo_df

,latitude,longitude,geometry
location_time,,,
2017-07-28 18:16:21.475999744,43.657532,-124.887015,POINT (-124.88702 43.65753)
2017-07-28 18:16:21.635000320,43.657500,-124.887000,POINT (-124.88700 43.65750)
2017-07-28 18:16:22.168999936,43.657532,-124.887080,POINT (-124.88708 43.65753)
2017-07-28 18:16:23.263000064,43.657532,-124.887147,POINT (-124.88715 43.65753)
2017-07-28 18:16:23.421000192,43.657500,-124.887167,POINT (-124.88717 43.65750)
...,...,...,...
2017-07-28 20:26:07.448999936,43.775167,-125.154667,POINT (-125.15467 43.77517)
2017-07-28 20:26:08.287000064,43.775265,-125.154647,POINT (-125.15465 43.77526)
2017-07-28 20:26:09.035999744,43.775310,-125.154635,POINT (-125.15463 43.77531)


### Create interface


- freq_checkbox

This function doesn't display names as intended, <a href="https://github.com/holoviz/panel/issues/1313">it's even discussed here</a>. So the workaround is to put everything into a column & give a descriptive name there. It does introduce an odd-looking text in comparison to built-in non-bug texts but it does its job. Might use something different to not ruin the overall look later. I'm thinking of using a dropdown menu instead. You can choose whichever looks more natural, I'm attaching both of them for comparison.

In [7]:
# create frequency checkbox to select only one value 
freq = pn.widgets.RadioBoxGroup(name='Select frequency',
                                options=[int(x) for x in files_mvbs\
                                         .frequency.values.tolist()])

freq_checkbox = pn.Column('##### Select frequency', freq)

freq_checkbox

Column
    [0] Markdown(str)
    [1] RadioBoxGroup(name='Select frequency', options=[18000, 38000, 120000], value=18000)

In [8]:
def show_freq(df, frequency=None):
    frequency = freq.value if frequency is None else frequency
    
    assert frequency in df.frequency, f'Frequency {frequency} is not in the dataset'
    
    df = df.assign_coords(depth=("range", df["range"].values[::-1]))
    df = df.swap_dims({'range': 'depth'})  # set depth as data dimension

    return df["Sv"].sel(frequency=frequency).hvplot.image(
            x='ping_time', y='depth', 
            color='Sv', rasterize=True, 
            cmap='jet', clim=(-80, -30),
            xlabel='Time (UTC)',
            ylabel='Depth (m)'
        ).options(width=800, invert_yaxis=True)

In [9]:
# example of how to use the function interactively
bind = pn.bind(show_freq, df=files_mvbs, frequency=freq)
bind()

:DynamicMap   []
   :Image   [ping_time,depth]   (Sv)

In [10]:
# alternate look
freq_checkbox2 = pn.widgets.Select(options=[int(x) for x in files_mvbs.frequency.values.tolist()], name='Select frequency')
freq_checkbox2

Select(options=[18000, 38000, 120000], value=18000)

### Date time selection

Also can use range: https://github.com/holoviz/panel/issues/1975

In [11]:
dt = pn.widgets.DatetimePicker(
    name='Select date/time',
    start=geo_df.index.min().date(),
    end=geo_df.index.max().date(),
    enable_time=True,
    enable_seconds=True,
    military_time=True
)

dt

DatetimePicker(end=datetime.date(2017, ..., name='Select date/time', start=datetime.date(2017, ...)

In [12]:
column = pn.Column('### Settings', 
                   dt,
                   freq_checkbox,
#                    freq_checkbox2,
                   background='WhiteSmoke')
column

Column(background='WhiteSmoke')
    [0] Markdown(str)
    [1] DatetimePicker(end=datetime.date(2017, ..., name='Select date/time', start=datetime.date(2017, ...)
    [2] Column
        [0] Markdown(str)
        [1] RadioBoxGroup(name='Select frequency', options=[18000, 38000, 120000], value=18000)

In [13]:
fig_dict, fig = get_map(geo_df)

In [14]:
pn.pane.Plotly(fig_dict, config={'responsive': True})

Plotly(dict, config={'responsive': True})

In [16]:
pn.Column(
    pn.Row(
        column,
        pn.Column(pn.Tabs(
            ('Default', pn.pane.Plotly(fig_dict, config={'responsive': True})),
            ('Results', pn.Tabs(
                      ('Result 1', fig_dict),
                      ('Result 2', fig_dict),
                      closable=True)))),
        margin=10
    ),
    
    pn.Row(
        pn.bind(show_freq, df=files_mvbs, frequency=freq),
        margin=(-20, 0, 0, 100)
    )
)

Column
    [0] Row(margin=10)
        [0] Column(background='WhiteSmoke')
            [0] Markdown(str)
            [1] DatetimePicker(end=datetime.date(2017, ..., name='Select date/time', start=datetime.date(2017, ...)
            [2] Column
                [0] Markdown(str)
                [1] RadioBoxGroup(name='Select frequency', options=[18000, 38000, 120000], value=18000)
        [1] Column
            [0] Tabs
                [0] Plotly(dict, config={'responsive': True})
                [1] Tabs(closable=True)
                    [0] Plotly(dict, name='Result 1')
                    [1] Plotly(dict, name='Result 2')
    [1] Row(margin=(-20, 0, 0, 100))
        [0] ParamFunction(function)